<a href="https://colab.research.google.com/github/NSR9/Room-Scene-recognition-and-Narration/blob/main/Room_Scene_recognition_and_Narration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyttsx3

In [3]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.0 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=7885628aa772d45dddd169a7d9887838d1dcfc3181fa7dc5a1430595c66473d1
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.0
    Uninstalling pyparsing-3.1.0:
      Successfully uninstalled pyparsing-3.1.0
  Attempting uninstall: idna
    Found e

In [6]:
! sudo apt-get install libespeak1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-data libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 4 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,158 kB of archives.
After this operation, 2,843 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 espeak-data amd64 1.48.04+dfsg-8build1 [932 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudio2 amd64 19.6.0-1build1 [65.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libsonic0 amd64 0.2.0-8 [13.1 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libespeak1 amd64 1.48.04+dfsg-8build1 [147 kB]
Fetched 1,158 kB in 2s (464 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so th

In [7]:
import time
import os

import pyttsx3
import roboflow

api_key = "CYzBf27ICRht54nH9vqB"
#rf = roboflow.Roboflow(api_key=os.environ["ROBOFLOW_API_KEY"])
rf = roboflow.Roboflow(api_key)


workspace = rf.workspace()

engine = pyttsx3.init()
engine.setProperty("rate", 150)

def say(text: str) -> None:
    """
    Speak out text and print that text to the console.
    """
    engine.say(text)
    print(text)

    engine.runAndWait()

def get_room_type(image_file: str) -> dict:
    """
    Get the type of room a photo was taken in (i.e. kitchen).
    """
    indoor_scene_recognition = workspace.project("mit-indoor-scene-recognition")

    model = indoor_scene_recognition.version(5).model

    prediction = model.predict(image_file)

    room = prediction.json()

    return room

def get_items_in_room(image: str) -> list:
    """
    Find items in a room.
    """
    project = workspace.project('all_finalize')

    model = project.version(3).model

    prediction = model.predict(image, confidence=10)

    predictions = prediction.json()

    labels = [p["class"] for p in predictions["predictions"]]

    prediction.save("out.png")

    return labels

def narrate_room(room_type: str, labels: list) -> None:
    """
    Speak out the type of room a photo was taken in and the objects in the room.
    """
    say(f"You are in a {room_type}")

    time.sleep(1)

    if len(labels) > 0:
        say("I see")
        time.sleep(0.5)

    for label in labels:
        label = label.lower()
        say("A " + label)

        time.sleep(0.5)

image = "kitchen1.jpeg"

room = get_room_type(image)
labels = get_items_in_room(image)

room_type = room["predictions"][0]["top"]

narrate_room(room_type, labels)

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow project...
You are in a kitchen
I see
A oven
A fridge
A cupboard
A exhaust hood
A stove
